In [22]:
import pandas as pd
import os
cwd = os.getcwd()
print(cwd)

/global/u2/a/alazar/fedLearn/src


In [80]:
feat=pd.read_csv('/global/cscratch1/sd/alazar/fedLearn/inputdata/features.csv',header=None)

In [81]:
checkd=feat.duplicated()
checkd
[i for i, x in enumerate(checkd) if x]

[]

In [82]:
feat=feat.T

In [83]:
feat

,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,149,150,151,152,153,154
0,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,...,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated1,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class


In [90]:
f = open('/global/cscratch1/sd/alazar/fedLearn/inputdata/features.csv', 'r')
feat = f.read().splitlines() 
f.close()

In [91]:
feat

['frame.interface_id',
 'frame.dlt',
 'frame.offset_shift',
 'frame.time_epoch',
 'frame.time_delta',
 'frame.time_delta_displayed',
 'frame.time_relative',
 'frame.len',
 'frame.cap_len',
 'frame.marked',
 'frame.ignored',
 'radiotap.version',
 'radiotap.pad',
 'radiotap.length',
 'radiotap.present.tsft',
 'radiotap.present.flags',
 'radiotap.present.rate',
 'radiotap.present.channel',
 'radiotap.present.fhss',
 'radiotap.present.dbm_antsignal',
 'radiotap.present.dbm_antnoise',
 'radiotap.present.lock_quality',
 'radiotap.present.tx_attenuation',
 'radiotap.present.db_tx_attenuation',
 'radiotap.present.dbm_tx_power',
 'radiotap.present.antenna',
 'radiotap.present.db_antsignal',
 'radiotap.present.db_antnoise',
 'radiotap.present.rxflags',
 'radiotap.present.xchannel',
 'radiotap.present.mcs',
 'radiotap.present.ampdu',
 'radiotap.present.vht',
 'radiotap.present.reserved',
 'radiotap.present.rtap_ns',
 'radiotap.present.vendor_ns',
 'radiotap.present.ext',
 'radiotap.mactime',
 'ra

In [96]:
# import a training set
awid = pd.read_csv("../data/AWID-CLS-R-Trn.csv", header=None, names=feat,low_memory=False)

# see the number of rows/columns
awid.shape

(1795575, 155)

In [97]:
# they use ? as a null attribute. We will eventually have to replace them with None values
awid.head()

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,...,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated1,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class
0,0,?,0.0,1.393661e+09,0.000000,0.000000,0.000000,261,261,0,...,?,?,?,?,?,?,?,?,?,normal
1,0,?,0.0,1.393661e+09,0.024271,0.024271,0.024271,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
2,0,?,0.0,1.393661e+09,0.001631,0.001631,0.025902,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
3,0,?,0.0,1.393661e+09,0.055325,0.055325,0.081227,159,159,0,...,?,?,?,?,?,?,?,?,?,normal
4,0,?,0.0,1.393661e+09,0.000415,0.000415,0.081642,54,54,0,...,?,?,?,?,?,?,?,?,?,normal


In [98]:
# see the distribution of response vars
awid['class'].value_counts(normalize=True)

normal           0.909564
injection        0.036411
impersonation    0.027023
flooding         0.027002
Name: class, dtype: float64

In [99]:
# claims there are no null values because of the ?'s'
awid.isna().sum()

frame.interface_id               0
frame.dlt                        0
frame.offset_shift               0
frame.time_epoch                 0
frame.time_delta                 0
                                ..
wlan.qos.bit4                    0
wlan.qos.txop_dur_req            0
wlan.qos.buf_state_indicated2    0
data.len                         0
class                            0
Length: 155, dtype: int64

In [100]:
# replace the ? marks with None
awid.replace({"?": None}, inplace=True)

In [101]:
# Many missing pieces of data!
awid.isna().sum()

frame.interface_id                     0
frame.dlt                        1795575
frame.offset_shift                     0
frame.time_epoch                       0
frame.time_delta                       0
                                  ...   
wlan.qos.bit4                    1648935
wlan.qos.txop_dur_req            1648935
wlan.qos.buf_state_indicated2    1279874
data.len                          903021
class                                  0
Length: 155, dtype: int64

In [102]:
# goal is to remove columns that have over 50% of missing data
columns_with_mostly_null_data = awid.columns[awid.isnull().mean() >= 0.5]

# 72 columns are going to be affected!
columns_with_mostly_null_data.shape

(72,)

In [103]:
# drop the columns with over half missing data
awid.drop(columns_with_mostly_null_data, axis=1, inplace=True)

In [104]:
awid.shape

(1795575, 83)

In [105]:
# now drop the rows that have missing values
awid.dropna(inplace=True)  # drop rows with null data

In [106]:
# lost 456,169 rows
awid.shape

(1339406, 83)

In [107]:
# doesn't affect our distribution too much

# 0.878763  is our null accuracy. Our model must be better than this number to be a contender
awid['class'].value_counts(normalize=True)

normal           0.878763
injection        0.048812
impersonation    0.036227
flooding         0.036198
Name: class, dtype: float64

In [108]:
# only select numerical columns for our ML algorithms, there should be more..
awid.select_dtypes(['number']).shape

(1339406, 45)

In [109]:
# transform all columns into numerical dtypes
for col in awid.columns:
    awid[col] = pd.to_numeric(awid[col], errors='ignore')

In [110]:
# that makes more sense
awid.select_dtypes(['number']).shape

(1339406, 74)

In [111]:
# basic descroptive statistics
awid.describe()

,frame.interface_id,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,frame.ignored,...,wlan.fc.frag,wlan.fc.retry,wlan.fc.pwrmgt,wlan.fc.moredata,wlan.fc.protected,wlan.fc.order,wlan.duration,wlan.frag,wlan.seq,wlan.fcs_good
count,1339406.0,1339406.0,1.339406e+06,1.339406e+06,1.339406e+06,1.339406e+06,1.339406e+06,1.339406e+06,1339406.0,1339406.0,...,1.339406e+06,1.339406e+06,1.339406e+06,1.339406e+06,1.339406e+06,1339406.0,1.339406e+06,1.339406e+06,1.339406e+06,1339406.0
mean,0.0,0.0,1.393663e+09,2.435200e-03,2.435200e-03,1.842787e+03,6.771728e+02,6.771728e+02,0.0,0.0,...,5.024615e-04,1.703964e-01,2.870825e-02,2.376427e-03,6.612961e-01,0.0,1.028715e+02,2.165885e-03,1.910773e+03,1.0
std,0.0,0.0,8.370012e+02,6.827733e-03,6.827733e-03,8.370012e+02,6.997003e+02,6.997003e+02,0.0,0.0,...,2.241003e-02,3.759808e-01,1.669853e-01,4.869067e-02,4.732692e-01,0.0,1.237048e+02,1.176732e-01,1.200198e+03,0.0
min,0.0,0.0,1.393661e+09,0.000000e+00,0.000000e+00,0.000000e+00,5.400000e+01,5.400000e+01,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.0
25%,0.0,0.0,1.393662e+09,4.430000e-04,4.430000e-04,1.103988e+03,9.800000e+01,9.800000e+01,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,4.400000e+01,0.000000e+00,8.410000e+02,1.0
50%,0.0,0.0,1.393663e+09,7.790000e-04,7.790000e-04,1.894319e+03,1.590000e+02,1.590000e+02,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0,4.400000e+01,0.000000e+00,1.812000e+03,1.0
75%,0.0,0.0,1.393664e+09,1.393000e-03,1.393000e-03,2.374825e+03,1.552000e+03,1.552000e+03,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0,2.580000e+02,0.000000e+00,2.941000e+03,1.0
max,0.0,0.0,1.393665e+09,3.502630e-01,3.502630e-01,3.599571e+03,1.570000e+03,1.570000e+03,0.0,0.0,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,3.200000e+02,1.200000e+01,4.095000e+03,1.0


In [112]:
X, y = awid.select_dtypes(['number']), awid['class']

In [139]:
import pickle
with open('/global/cscratch1/sd/alazar/fedLearn/pfile/train.pk', 'wb') as f:
    pickle.dump([X,y], f)

In [114]:
# do a basic naive bayes fitting
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

# fit our model to the data
nb.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [116]:
# read in the test data and do the same transformations to it to match the training data
awid_test = pd.read_csv('/global/cscratch1/sd/alazar/fedLearn/inputdata/AWID-CLS-R-Tst.csv', header=None, names=feat,low_memory=False)
# drop the problematic columns
awid_test.drop(columns_with_mostly_null_data, axis=1, inplace=True)
# replace ? with None
awid_test.replace({"?": None}, inplace=True)
# drop the rows with null data
awid_test.dropna(inplace=True)  # drop rows with null data
# convert columns to numerical values
for col in awid_test.columns:
    awid_test[col] = pd.to_numeric(awid_test[col], errors='ignore')
awid_test.shape

(389185, 83)

In [117]:
awid_test.head()

,frame.interface_id,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,frame.ignored,...,wlan.duration,wlan.ra,wlan.da,wlan.ta,wlan.sa,wlan.bssid,wlan.frag,wlan.seq,wlan.fcs_good,class
0,0,0.0,1.393668e+09,0.000000,0.000000,0.000000,1552,1552,0,0,...,44,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0,2313,1,normal
1,0,0.0,1.393668e+09,0.001056,0.001056,0.001056,1552,1552,0,0,...,44,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0,2314,1,normal
4,0,0.0,1.393668e+09,0.001842,0.001842,0.003195,1552,1552,0,0,...,44,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0,2318,1,normal
5,0,0.0,1.393668e+09,0.006015,0.006015,0.009210,1552,1552,0,0,...,44,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0,2328,1,normal
6,0,0.0,1.393668e+09,0.000940,0.000940,0.010150,1552,1552,0,0,...,44,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0,2330,1,normal


In [118]:
# basic metric, accuracy
from sklearn.metrics import accuracy_score

In [119]:
X_test = awid_test.select_dtypes(['number'])
y_test = awid_test['class']

# simple function to test the accuracy of a model fitted on training data on our testing data
def get_test_accuracy_of(model):
    y_preds = model.predict(X_test)
    return accuracy_score(y_preds, y_test)
    
# naive abyes does very poorly on its own!
get_test_accuracy_of(nb)

0.26535452291326744

In [140]:
with open('/global/cscratch1/sd/alazar/fedLearn/pfile/test.pk', 'wb') as f:
    pickle.dump([X_test,y_test], f)

In [120]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X, y)

# Logistic Regressions does even worse
get_test_accuracy_of(lr)

/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.015773989233911892

In [121]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()

tree.fit(X, y)

# Tree does very well!
get_test_accuracy_of(tree)

0.934023664838059

In [122]:
# see gini scores of decision tree features
pd.DataFrame({'feature':awid.select_dtypes(['number']).columns, 
              'importance':tree.feature_importances_}).sort_values('importance', ascending=False).head(10)

,feature,importance
4,frame.time_delta_displayed,0.221155
6,frame.len,0.197393
68,wlan.fc.protected,0.145766
70,wlan.duration,0.127600
7,frame.cap_len,0.092752
5,frame.time_relative,0.082126
62,wlan.fc.type,0.040173
72,wlan.seq,0.026933
65,wlan.fc.retry,0.019813
58,radiotap.dbm_antsignal,0.014189


In [123]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()

forest.fit(X, y)

# Random Forest does slightly worse
get_test_accuracy_of(forest)

/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9352955535285276

In [124]:
# Create a pipeline that will scale the numerical data and then feed the resulting data into a decision tree

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

preprocessing = Pipeline([
    ("scale", StandardScaler()),
])

pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("classifier", DecisionTreeClassifier())
])

# try varying levels of depth
params = {
    "classifier__max_depth": [None, 3, 5, 10], 
         }

# instantiate a gridsearch module
grid = GridSearchCV(pipeline, params)
# fit the module
grid.fit(X, y)

# test the best model
get_test_accuracy_of(grid.best_estimator_)

/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.9254621837943394

In [125]:
# try the same thing with a random forest

preprocessing = Pipeline([
    ("scale", StandardScaler()),
])

pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("classifier", RandomForestClassifier())
])

# try varying levels of depth
params = {
    "classifier__max_depth": [None, 3, 5, 10], 
         }

grid = GridSearchCV(pipeline, params)
grid.fit(X, y)
# best accuracy so far!
get_test_accuracy_of(grid.best_estimator_)

/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/global/homes/a/alazar/.conda/envs/fedenv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
 

0.8992021789123424

In [126]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded_y = encoder.fit_transform(y)
encoded_y.shape

(1339406,)

In [127]:
encoded_y

array([3, 3, 3, ..., 3, 3, 3])

In [128]:
from sklearn.preprocessing import LabelBinarizer
binarizer = LabelBinarizer()
binarized_y = binarizer.fit_transform(encoded_y)
binarized_y.shape

(1339406, 4)

In [129]:
binarized_y[:5,]

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [130]:
y.head()

0    normal
1    normal
2    normal
3    normal
4    normal
Name: class, dtype: object

In [132]:
print (encoder.classes_)
print (binarizer.classes_)

['flooding' 'impersonation' 'injection' 'normal']
[0 1 2 3]


In [133]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

ModuleNotFoundError: No module named 'keras'

In [136]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'